# 1. Azure ML Workspace Configurarion

In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-123396")
#exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

#run = exp.start_logging()

Workspace name: quick-starts-ws-123396
Azure region: southcentralus
Subscription id: 4910dccd-0348-46c4-a51f-d8c85e078b14
Resource group: aml-quickstarts-123396


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
amlcompute_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    aml_compute = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    aml_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


# 2. Hyperparameter optimization using Hyperdrive

In [14]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive import choice
from azureml.train.hyperdrive.parameter_expressions import uniform
import os
from azureml.train.estimator import Estimator

# Specify parameter sampler
### YOUR CODE HERE ###
ps = RandomParameterSampling(
    {
        '--C': uniform(0.1, 3),
        '--max_iter': choice([50, 100, 200])
    }
)

# Specify a Policy
### YOUR CODE HERE ###
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

script_params = {
    '--data_folder': ws.get_default_datastore(),
}

est = Estimator(source_directory='.',
                script_params=script_params,
                compute_target=aml_compute,
                entry_script='train.py',
                pip_packages=['scikit-learn']
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(
    estimator=est,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=16,
    max_concurrent_runs=4)

In [15]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
from azureml.widgets import RunDetails
experiment = Experiment(workspace=ws, name='LogReg-hyperdrive')
hyperdrive_run = experiment.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.
WARNING - If 'arguments' has been provided here and arguments have been specified in 'run_config', 'arguments' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [32]:
import pickle
from multiprocessing import Queue
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
print(best_run)
print('Best accuracy: {}'.format(best_run_metrics['Accuracy']))

os.makedirs('outputs',exist_ok=True)
model_name = "best_run.pkl"
filename = "outputs/" + model_name
joblib.dump(value=best_run, filename=model_name)

Run(Experiment: LogReg-hyperdrive,
Id: HD_07e58109-89f6-45f5-86a8-96f6fe7ae94f_7,
Type: azureml.scriptrun,
Status: Completed)
Best accuracy: 0.9153920854576354


# 3. AutoML

In [33]:
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import DataType
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
web_path = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
ds = Dataset.Tabular.from_delimited_files(path = web_path)

In [35]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [36]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='AUC_weighted',
    training_data=x.join(y),
    label_column_name="y",
    n_cross_validations=5)

In [37]:
# Submit your automl run

### YOUR CODE HERE ###
experiment = Experiment(workspace=ws, name='bankmarketing-automl')
automl_run = experiment.submit(automl_config, show_output = False)

WARNING - Running on local machine. Note that local runs always run synchronously even if you use the parameter 'show_output=False'


In [38]:
automl_run.wait_for_completion()

{'runId': 'AutoML_c69bd376-96ee-4bde-9662-84c3726e163e',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2020-10-27T20:42:14.534593Z',
 'endTimeUtc': '2020-10-27T21:16:12.672206Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"bankmarketing-automl","subscription_id":"54424c07-8d8d-4896-9c92-b731a63208d3","resource_group":"aml-quickstarts-123258","workspace_name":"quick-starts-ws-123258","region":"southcentralus","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","_local_managed_run_id":null,"many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"AUC_weighted","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y

In [40]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run_automl, fitted_model_automl = automl_run.get_output()

In [41]:
best_run_automl

model_name = "best_run_automl.pkl"
filename = "outputs/" + model_name
joblib.dump(value=best_run_automl, filename=model_name)

Experiment,Id,Type,Status,Details Page,Docs Page
bankmarketing-automl,AutoML_c69bd376-96ee-4bde-9662-84c3726e163e_54,,Completed,Link to Azure Machine Learning studio,Link to Documentation


# 4. Cluster Cleanup

### List some details

In [43]:
aml_compute.name

'cpu-cluster'

In [44]:
print(ws.get_default_compute_target('CPU'))

WARNING - 'get_default_compute_target' is deprecated and will be removed in a future release. Use compute_targets['name'] to retrieve an existing compute target instead.


AmlCompute(workspace=Workspace.create(name='quick-starts-ws-123396', subscription_id='4910dccd-0348-46c4-a51f-d8c85e078b14', resource_group='aml-quickstarts-123396'), name=cpu-cluster, id=/subscriptions/4910dccd-0348-46c4-a51f-d8c85e078b14/resourceGroups/aml-quickstarts-123396/providers/Microsoft.MachineLearningServices/workspaces/quick-starts-ws-123396/computes/cpu-cluster, type=AmlCompute, provisioning_state=Succeeded, location=southcentralus, tags=None)


In [45]:
aml_compute.get_status()

### Clean up compute cluster

In [46]:
aml_compute.delete()

In [47]:
print(ws.get_default_compute_target('CPU'))

WARNING - 'get_default_compute_target' is deprecated and will be removed in a future release. Use compute_targets['name'] to retrieve an existing compute target instead.


None
